In [5]:
import yfinance as yf
import numpy as np
import pandas as pd
import scipy.optimize as opt
from chapter_6 import Strategy, Engine

In [6]:
START_DATE_DATA = "2023-07-01"
START_DATE_REBALANCE = "2024-01-01"
END_DATE = "2024-12-31"
ASSETS = ["AAPL", "MSFT", "GOOGL", "AMZN", "BRK-B"]

data = {
    asset: yf.Ticker(asset)
    .history(
        start=START_DATE_DATA,
        end=END_DATE) 
    for asset in ASSETS
    }
 
data = {
    asset: data[asset].tz_localize(None)
    for asset in ASSETS
    }
 
returns = pd.DataFrame(
    {
        asset: data[asset]["Close"]
        for asset in ASSETS
     }).pct_change().dropna()


In [7]:
def sharpe_ratio(weights, ret_subset):

    mean_returns = ret_subset.mean() * 252
    cov_returns = ret_subset.cov() * 252

    port_ret = np.dot(
        weights,
        mean_returns
        )    
    port_vol = np.sqrt(
        np.dot(
            weights,
            np.dot(
                cov_returns, weights)
            )
        )
    return -port_ret / port_vol


In [8]:
# DECLARE PARAMETERS
NUM_ASSETS = len(ASSETS)
CONSTRAINTS = {
    "type": "eq",
    "fun": lambda x: np.sum(x) - 1
    }
BOUNDARIES = [(0.1, 1.0)] * NUM_ASSETS
INITIAL_WEIGHTS = np.full(
    NUM_ASSETS, 1.0 / NUM_ASSETS
    )
REBALANCE_DATES = pd.date_range(
    start=START_DATE_REBALANCE,
    end=END_DATE,
    freq="MS"
)
 
# RUN OPTIMIZATION
optimal_weights = {}
for date in REBALANCE_DATES:
    lookback_start = date - pd.DateOffset(months=6)
    ret_subset = returns.loc[
        lookback_start : date - pd.Timedelta(days=1)
        ]
    opt_result = opt.minimize(
        sharpe_ratio,
        INITIAL_WEIGHTS,
        args=(ret_subset,),
        method="SLSQP",
        bounds=BOUNDARIES,
        constraints=CONSTRAINTS,
    )
    if opt_result.success:
        optimal_weights[date] = dict(
            zip(ASSETS, opt_result.x)
        )
 


In [ ]:
class RebalanceExposure(Strategy):
    def __init__(self, weights):
        super().__init__()
        self.weights = pd.DataFrame.from_dict(weights).T
        # LAST REBALANCE MONTH
        self.lbm = 0
 
    def on_bar(self):
        if  (
            self.lbm != self.current_idx.month
            ):
            weights = (self.weights
                       .loc[:self.current_idx]
                       .iloc[-1]
                       )
            self.rebalance(weights)
            self.lbm = self.current_idx.month
 
 
e = Engine()
for ticker in data:
    prices = data[ticker].loc[START_DATE_REBALANCE:]
    e.add_data(prices, ticker)
 
e.add_strategy(
    RebalanceExposure(
        weights=optimal_weights
        )
    )
out = e.run()
print('Ann. Returns [%]: ',round(out['annualized_returns'],2))
print('Ann. Volatility [%]: ',round(out['volatility'],2))
print('Sharpe Ratio: ',round(out['sharpe_ratio'],2))
print('Max Drawdown [%]: ',round(out['max_drawdown'],2))
print('Exposure [%]: ',out['exposure'])


100%|██████████| 251/251 [00:00<00:00, 7306.79it/s]

Ann. Returns [%]:  31.01
Ann. Volatility [%]:  14.19
Sharpe Ratio:  1.91
Max Drawdown [%]:  -9.01
Exposure [%]:  98.99803454977366
Trade Journal: 


In [12]:
class EqualWeight(Strategy):
    def __init__(self):
        super().__init__()
        self.lbm = 0
 
    def on_bar(self):
        if  (
            self.lbm != self.current_idx.month
            ):
            weights = {
                ticker: 1 / len(self.tickers) 
                for ticker in self.tickers
            } 
            self.rebalance(weights)
            self.lbm = self.current_idx.month

e = Engine()
for ticker in data:
    prices = data[ticker].loc[START_DATE_REBALANCE:]
    e.add_data(prices, ticker)
 
e.add_strategy(
    EqualWeight()
    )
out = e.run()
print('Ann. Returns [%]: ',round(out['annualized_returns'],2))
print('Ann. Volatility [%]: ',round(out['volatility'],2))
print('Sharpe Ratio: ',round(out['sharpe_ratio'],2))
print('Max Drawdown [%]: ',round(out['max_drawdown'],2))
print('Exposure [%]: ',out['exposure'])

100%|██████████| 251/251 [00:00<00:00, 7359.35it/s]

Ann. Returns [%]:  35.03
Ann. Volatility [%]:  16.58
Sharpe Ratio:  1.82
Max Drawdown [%]:  -12.3
Exposure [%]:  99.02805840949998
